**Animal Performance on W-Track** 

**Set up**

In [3]:
data_directory = '/home/zoldello/Documents/data/L5/r1/20230531_150338/'
trodes_directory = '/home/zoldello/Apps/Trodes_2-4-0_Ubuntu2004/'

file_name = f'{data_directory}20230531_150338.rec'
trodesexport_runner = f'{trodes_directory}trodesexport'

In [4]:
!$trodesexport_runner -rec {file_name} -dio -time
#!/home/zoldello/Apps/Trodes_2-4-0_Ubuntu2004/trodesexport -rec {file_name} -dio -time

/bin/bash: line 1: /home/zoldello/Apps/Trodes_2-4-0_Ubuntu2004/trodesexport: No such file or directory


In [5]:
#trodes_directory
trodesexport_runner

'/home/zoldello/Apps/Trodes_2-4-0_Ubuntu2004/trodesexport'

**Trodes' Functions**

In [ ]:
import numpy as np
import re
from sys import argv
# Main function
def readTrodesExtractedDataFile(filename):
    with open(filename, 'rb') as f:
        # Check if first line is start of settings block
        if f.readline().decode('ascii').strip() != '<Start settings>':
            raise Exception("Settings format not supported")
        fields = True
        fieldsText = {}
        for line in f:
            # Read through block of settings
            if(fields):
                line = line.decode('ascii').strip()
                # filling in fields dict
                if line != '<End settings>':
                    vals = line.split(': ')
                    fieldsText.update({vals[0].lower(): vals[1]})
                # End of settings block, signal end of fields
                else:
                    fields = False
                    dt = parseFields(fieldsText['fields'])
                    fieldsText['data'] = np.zeros([1], dtype = dt)
                    break
        # Reads rest of file at once, using dtype format generated by parseFields()
        dt = parseFields(fieldsText['fields'])
        data = np.fromfile(f, dt)
        fieldsText.update({'data': data})
        return fieldsText


# Parses last fields parameter (<time uint32><...>) as a single string
# Assumes it is formatted as <name number * type> or <name type>
# Returns: np.dtype
def parseFields(fieldstr):
    # Returns np.dtype from field string
    sep = re.split('\s', re.sub(r"\>\<|\>|\<", ' ', fieldstr).strip())
    # print(sep)
    typearr = []
    # Every two elmts is fieldname followed by datatype
    for i in range(0,sep.__len__(), 2):
        fieldname = sep[i]
        repeats = 1
        ftype = 'uint32'
        # Finds if a <num>* is included in datatype
        if sep[i+1].__contains__('*'):
            temptypes = re.split('\*', sep[i+1])
            # Results in the correct assignment, whether str is num*dtype or dtype*num
            ftype = temptypes[temptypes[0].isdigit()]
            repeats = int(temptypes[temptypes[1].isdigit()])
        else:
            ftype = sep[i+1]
        try:
            fieldtype = getattr(np, ftype)
        except AttributeError:
            print(ftype + " is not a valid field type.\n")
            exit(1)
        else:
            typearr.append((str(fieldname), fieldtype, repeats))
    return np.dtype(typearr)

**Dins**

In [ ]:
din_path = ''
Din1 = readTrodesExtractedDataFile(f'{din_path}/20230502_163824.dio_ECU_Din1.dat')
Din2 = readTrodesExtractedDataFile(f'{din_path}/20230502_163824.dio_ECU_Din2.dat')
Din7 = readTrodesExtractedDataFile(f'{din_path}/20230502_163824.dio_ECU_Din7.dat')

**Dout**

In [ ]:
dout_path = ''
Dout4 = readTrodesExtractedDataFile(f'{dout_path}/20230502_163824.dio_ECU_Din4.dat')
Dout5 = readTrodesExtractedDataFile(f'{dout_path}/20230502_163824.dio_ECU_Din5.dat')
Dout6 = readTrodesExtractedDataFile(f'{dout_path}/20230502_163824.dio_ECU_Din6.dat')

**Time**

In [ ]:
time = readTrodesExtractedDataFile('/stelmo/kyu/L5/20230502/r2/20230502_163824/20230502_163824.time/20230502_163824.timestamps.dat')